In [6]:
%%time

import pickle
import numpy as np
import pandas as pd

from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
from collections import Counter

from html.parser import HTMLParser
from collections import OrderedDict
import re
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from bs4 import BeautifulSoup

#############################
#
# setup # look at http://pythondata.com/category/text-analytics/
#
#############################

porter = PorterStemmer()
wnl = WordNetLemmatizer()
stop = stopwords.words('german')
#stop.append("blabla")
stop = set(stop)

#############################
#
# load the data
#
#############################

with open('cleaned_sentences.pickle', 'rb') as handle:
    cleaned_sentences = pickle.load(handle)
    
    
#############################
#
# tokenize the text # look at http://pythondata.com/category/text-analytics/
#
#############################

def tokenize(text):
    
    tokens_ = [word_tokenize(sent) for sent in sent_tokenize(text)]
    
    tokens = []
    for token_by_sent in tokens_:
        tokens += token_by_sent
        
    tokens = list(filter(lambda t: t.lower() not in stop, tokens))
    tokens = list(filter(lambda t: t not in punctuation, tokens))
    tokens = list(filter(lambda t: t not in [u"'s",u"n't", u"...", u"''", u'``', u'\u2014', u'\u2026', u'\u2013'], tokens))
    
    filtered_tokens = []
    for token in tokens:
        token = wnl.lemmatize(token)
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    
    filtered_tokens = list(map(lambda token: token.lower(), filtered_tokens))
    
    return filtered_tokens

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

def get_keywords(tokens, num):
    return Counter(tokens).most_common(num)

def build_article_df(text):
    articles = []
    for index, row in text.iterrows():
        try:
            data=row['text'].strip().replace("'", "")
            data = strip_tags(data)
            soup = BeautifulSoup(data)
            data = soup.get_text()
            data = data.encode('ascii', 'ignore').decode('ascii')
            document = tokenizer(data)
            top_5 = get_keywords(document, 5)
            
            unzipped = zip(*top_5)
            kw= list(unzipped[0])
            kw=",".join(str(x) for x in kw)
            articles.append((kw, row['Index'], ['Text']))
        except Exception as e:
            print(e)
            #print data
            #break
            pass
        #break
    article_df = pd.DataFrame(articles, columns=['keywords', 'Text', 'Index'])
    return article_df

#df = pd.read_csv('../examples/tocsv.csv')
df = cleaned_sentences

data = []
for index, row in df.iterrows():
    data.append((row['Index'], row['Sentence'], row['Book_ID']))
data_df = pd.DataFrame(data, columns=['Index' ,'Text', 'Book'])


# build keywords for every article
article_df = build_article_df(data_df) 
article_df.head()


"""
keywords_array=[]
for index, row in article_df.iterrows():
    keywords=row['keywords'].split(',')
    for kw in keywords:
        keywords_array.append((kw.strip(' '), row['keywords']))
kw_df = pd.DataFrame(keywords_array).rename(columns={0:'keyword', 1:'keywords'})

document = kw_df.keywords.tolist()
names = kw_df.keyword.tolist()
 
document_array = []
for item in document:
    items = item.split(',')
    document_array.append((items))
 
occurrences = OrderedDict((name, OrderedDict((name, 0) for name in names)) for name in names)
 
# Find the co-occurrences:
for l in document_array:
    for i in range(len(l)):
        for item in l[:i] + l[i + 1:]:
            occurrences[l[i]][item] += 1
 
co_occur = pd.DataFrame.from_dict(occurrences )

# export to csv
co_occur.to_csv('out/ericbrown_co-occurancy_matrix.csv')
"""



KeyError: 'Index'